In [1]:
library(tidyverse)
theme_set(theme_classic())
options(repr.plot.width=10, repr.plot.height=5)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# STATS 504
## Week 5: NHANES data

## The data

- [**NHANES**](https://www.cdc.gov/nchs/nhanes/index.htm) stands for the National Health and Nutrition Examination Survey.
- Series of studies designed to assess the health and nutritional status of adults and children in the United States.
- Conducted by the National Center for Health Statistics (NCHS), part of the Centers for Disease Control and Prevention (CDC).

## Purpose of NHANES

- To determine the prevalence of major diseases and risk factors for diseases.
- To assess nutritional status and its association with health promotion and disease prevention.
- To guide public health policy and priorities.



## How NHANES Works

- NHANES combines interviews and physical examinations.
- The interview includes demographic, socioeconomic, dietary, and health-related questions.
- The examination part includes medical, dental, and physiological measurements, as well as laboratory tests administered by medical personnel.
- NHANES data is collected in 2-year cycles and released in public-use data files.

## Questions we will study today
1. Is height associated with income? (Does height *cause* you to have higher income?)
2. Is smoking associated with high blood pressure (hypertension)?
3. How does TV watching affect body weight?

For today's lecture we will use the following "pre-packaged" NHANES extracted dataset:

In [2]:
# install.packages('NHANES')
library(NHANES)
data(NHANES)

help(NHANES)

(Next lecture, we will see how to import the raw data directly off their website.)

In [4]:
?NHANES

## Height vs. income
It is often claimed that taller people earn more money. Do we see this in the data?

## Quiz 🤷  

Comparing the mean height of the highest income level vs. everyone else, we find that:

<ol style="list-style-type: upper-alpha;">
    <li> Mean height among the richest individuals is significantly <emph>higher</emph> compared to the rest.
    <li> Mean height among the richest individuals is significantly <emph>lower</emph> compared to the rest..
    <li> Mean height among the richest individuals is <b>in</b>significantly <emph>higher</emph> compared to the rest.
    <li> Mean height among the richest individuals is <b>in</b>significantly <emph>lower</emph> compared to the rest.</ol>

"Significant" means statistically significant at the $\alpha=0.05$ level.

Let's try stratifying income into quartiles:

## Digression: association vs. causation
- The preceding analyses reveal that there is strong *association* between income and height.
- Does that mean that being taller *causes you* to have higher income? (Why or why not?)
- (We will return to these points in a future lecture.)

## Is blood pressure elevated in smokers?

- Smoking tobacco increases heart rate and blood pressure. Long-term smoking can lead to the development of hypertension and other cardiovascular diseases.
- Nicotine from cigarettes stimulates the body to produce adrenaline, which accelerates the heart rate and raises blood pressure. Other chemicals in tobacco can damage the arterial walls, contributing to heart conditions.
- Some studies have shown a correlation between smoking and increased risk of developing hypertension. This relationship is dose-dependent, with heavier smoking leading to greater risks.
- Do we see such a relationship in the NHANES data?

It's visually hard for me to distinguish these two distributions. 

## Quiz 🤷  

Comparing the mean systolic blood pressure of the smokers (people who answered yes to `SmokeNow`) vs. non-smokers, we find that:

<ol style="list-style-type: upper-alpha;">
    <li> Mean systolic bp among smokers is significantly <emph>higher</emph> than among nonsmokers.
    <li> Mean systolic bp among smokers is significantly <emph>lower</emph> than among nonsmokers.
    <li> Mean systolic bp among smokers is <b>in</b>significantly <emph>higher</emph> than among nonsmokers.
    <li> Mean systolic bp among smokers is <b>in</b>significantly <emph>lower</emph> than among nonsmokers.
</ol>

"Significant" means statistically significant at the $\alpha=0.05$ level.

- We get the (possibly) counter-intuitive result that blood pressure is lower in smokers. 
- What are possible explanations for this?

There may be potential confounders! 


## 1. Age Confounding

- Smokers may be younger than nonsmokers in population datasets like NHANES. (need to verify)
- Blood pressure increases with age so younger age lowers the average SBP in smokers.

## 2. BMI and Body Composition

- Smokers may have lower body weight or BMI than nonsmokers.
- Lower BMI is associated with lower blood pressure.
  

Let's try looking at this through a different lens. The American Heart Association defines hypertension as systolic blood pressure > 130 mm Hg or diastolic bp > 80 mm Hg. Let's try logistic regression on hypertensive status.

In fact there are four stages of hypertension:
- **Normal Blood Pressure**
  - Systolic: Less than 120 mm Hg
  - Diastolic: Less than 80 mm Hg
- **Elevated Blood Pressure**
  - Systolic: 120-129 mm Hg
  - Diastolic: Less than 80 mm Hg
- **Hypertension Stage 1**
  - Systolic: 130-139 mm Hg
  - Diastolic: 80-89 mm Hg
- **Hypertension Stage 2**
  - Systolic: At least 140 mm Hg
  - Diastolic: At least 90 mm Hg
- **Hypertensive Crisis (Emergency situation)**
  - Systolic: Over 180 mm Hg and/or
  - Diastolic: Over 120 mm Hg

## Effects at the tails
Let's consider whether there is a measurable effect for stage 2 hypertension:

## Ordinal logistic regression

Ordinal Logistic Regression, also known as Ordered Logit Regression, is a statistical technique used for modeling the relationship between an ordinal dependent variable and one or more independent variables.

- Ordinal dependent variable: A categorical variable with a clear ordering of the categories, but not necessarily a constant difference between categories.
- Independent variables: Can be continuous, dichotomous, or categorical, and are used to predict the ordinal outcome.


### OLR model
$$
\log\left(\frac{P(Y \leq j)}{P(Y > j)}\right) = \alpha_j - \beta X, \quad j = 1, 2, \ldots, J-1
$$

where:

- $Y$ is the ordinal dependent variable with $J$ ordered categories.
- $P(Y \leq j)$ is the cumulative probability of $Y$ being in category $j$ or lower.
- $X$ represents the vector of independent variables (predictors).
- $\beta$ is the vector of coefficients associated with the predictors.
- $\alpha_j$ are the threshold parameters (cutpoints) for each $j$th category, with $j = 1, 2, \ldots, J-1$.

### Proportional odds assumption
The model implies the following key assumption:
$$
\frac{\partial}{\partial X}\log\left(\frac{P(Y \leq j)}{P(Y > j)}\right) = -\beta, \quad \text{for all } j
$$

This says that the odds ratios comparing any two outcome categories are assumed to be the same across all levels of the predictors.

In [23]:
bp_df$BPStage <- factor(
    bp_df$BPStage, ordered = TRUE, 
     levels = c("Normal", "Elevated", "Hypertension Stage 1", 
                "Hypertension Stage 2", "Hypertensive Crisis")
    )

### Interpretation

Important to distinguish between the coefficients and the odds ratios:

- **Coefficients**: Represent the log odds of being in a higher category of the outcome variable for a one-unit increase in the predictor.
- **Odds Ratios**: Exponentiated coefficients, indicating how the odds of being in a higher category change with a one-unit increase in the predictor.

Hence:

- **Positive Coefficient (Odds Ratio > 1)**: Indicates an increase in the predictor is associated with higher odds of being in a higher category.
- **Negative Coefficient (Odds Ratio < 1)**: Suggests an increase in the predictor is associated with lower odds of being in a higher category.

## TV watching and and body mass
How does watching TV associate with body weight among individuals in the NHANES dataset? We'll study this in several ways:

- **Descriptive Analysis**: Understand the distribution of TV and BMI.
- **Correlation Analysis**: Examine the linear relationship between TV and BMI.
- **Linear Regression**: Assess the impact of TV on BMI, controlling for confounders.
- **Logistic Regression**: Evaluate the odds of being overweight or obese with increased TV watching.
- **Mixed Models/GEE**: Account for clustering within the data.
- **Non-Linear Models**: Explore potential non-linear relationships between TV and BMI.

### Descriptive analysis
We start by considering the distribution of BMI. The WHO defines BMI according to the following levels:
- Underweight: BMI less than 18.5+
- Normal weight: BMI 18.5 to 24.9
- Overweight: BMI 25 to 29.9
- Obesity: BMI 30 and above

## Regression analysis
Next, let's model the effect of TV watching on BMI. We'll try two different ways:

Next, let's try with the categorized `TVHrsDay` replaced by a continuous estimate:

### Effect on obesity
Let's consider the effect on obesity (which is defined above as BMI > 30).

How can we use the logistic regression model to give a "predicted probability of obesity"?

### Non-linear model

Generalized Additive Models (GAMs) extend linear models by allowing non-linear relationships between the independent variables and the dependent variable through the use of smooth functions.
- A GAM can be expressed as $y = \beta_0 + f_1(x_1) + f_2(x_2) + \cdots + f_n(x_n) + \epsilon$, where $y$ is the dependent variable, $\beta_0$ is the intercept, $f_i$ are smooth functions for each predictor $x_i$, and $\epsilon$ is the error term.
- In our example, we'll suppose that 

$$\text{BMI} = s(\text{Age}) + \text{other predictors},$$

where $s()$ denotes a smoothing function applied to age.